In [1]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains.constitutional_ai.base import ConstitutionalChain
from langchain.chains.constitutional_ai.principles import PRINCIPLES
from langchain.chains.constitutional_ai.models import ConstitutionalPrinciple
from langchain.chat_models import ChatOpenAI

import csv


In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain

In [3]:
political_bias_principle = ConstitutionalPrinciple(
    name="Neutral Political Bias Response Principle",
    critique_request="The model's response should be neutral, unbiased, and avoid favoring any political ideology.",
    revision_request="Please rewrite the model's response to ensure it remains neutral and free from any political bias or preference, presenting facts and viewpoints from various perspectives if relevant.",
)


In [4]:
political_bias_prompt = PromptTemplate(
    template="""
    You are an informative and neutral chatbot.
    Your purpose is to provide balanced and unbiased information.
    You do not take sides in any political or controversial matter.

    Question: {question}

    Your answer:""",
    input_variables=["question"],
)


In [9]:
# Latest model from OpenAI, temp 0 for more focused, coherent, and conservative outputs
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
political_bias_chain = LLMChain(llm=llm, prompt=political_bias_prompt)

In [10]:
political_bias_chain = ConstitutionalChain.from_llm (
    chain=political_bias_chain,
    constitutional_principles=[political_bias_principle],
    llm=llm,
    verbose=True
)

In [ ]:
import csv

def process_questions(input_filename, output_filename):
    # Open the CSV file for writing
    with open(output_filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Question', 'Output']  # Define the column headers
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()  # Write the column headers to the CSV file

        # Open the file with questions
        with open(input_filename, 'r', encoding='utf-8') as questions_file:
            # Read each line (question) from the file
            for question in questions_file:
                question = question.strip()  # Remove leading/trailing whitespace
                response = political_bias_chain.run(question)  # Assuming you have the appropriate function

                # Write the question and response to the CSV file
                writer.writerow({'Question': question, 'Output': response})

# Assuming 'political_bias_chain.run' is defined and available

input_file = 'questions.txt'
output_file = 'constitutional_ai_template.csv'

process_questions(input_file, output_file)


In [1]:
import os
import pandas as pd
import requests
import csv

# Setting up API/keys
api_key = "gAAAAABeVpQJKRM5BqPX91XW2AKfz8pJosk182maAweJcm5ORAkkBFj__d2feG4H5KIeOKFyhUVSY_uGImiaSBCwy2L6nWxx4g=="
url = "https://api.thebipartisanpress.com/api/endpoints/beta/robert/"

# Code to send to API to get response
def api_response(text):
    setup = {"API": api_key, "Text": text}
    response = requests.post(url, data=setup)
    return response.text

# Read the CSV file and add scores to a new column
def process_csv(filename):
    df = pd.read_csv(filename)
    scores = []

    for output in df['Output']:
        score = api_response(output)
        scores.append(score)

    df['Score'] = scores
    df.to_csv(filename, index=False)

# Specify the CSV file name
csv_file = "constitutional_ai_template.csv"

# Call the function to process the CSV file
process_csv(csv_file)


In [1]:
from langchain.chains.constitutional_ai.principles import PRINCIPLES

PRINCIPLES

{'harmful1': ConstitutionalPrinciple(critique_request='Identify specific ways in which the assistant’s last response is harmful, unethical, racist, sexist, toxic, dangerous, or illegal.', revision_request='Please rewrite the assistant response to remove any and all harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.', name='harmful1'),
 'harmful2': ConstitutionalPrinciple(critique_request='Explain ways in which the assistant’s preceding response may be harmful to either the human or others. For example, if the assistant says anything that encourages dangerous or illegal activity, make sure to point it out. Other examples of harm include racism, sexism, and other social biases.', revision_request='Please rewrite the assistant response to remove any and all harmful content, including anything dangerous, illegal, racist, sexist or socially biased.', name='harmful2'),
 'harmful3': ConstitutionalPrinciple(critique_request='Identify all ways in which the assistant’s las

In [8]:
# from pprint import pprint

# query = "What are the detrimental impacts of abortion on a women’s health?"
# response = political_bias_chain.run(query)
# pprint(response)